In [ ]:
import torch
import torch.nn as nn
import numpy as np
from gensim.models import KeyedVectors
from main import run

In [ ]:
def load_glove_embeddings(file_path, word_to_index, embedding_dim):
    embeddings = {}
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype="float32")
            embeddings[word] = coefs

    num_words = len(word_to_index) + 1  # add 1 for the padding token
    embedding_matrix = np.zeros((num_words, embedding_dim))

    for word, index in word_to_index.items():
        if (embedding := embeddings.get(word)) is not None:
            embedding_matrix[index] = embedding
        else:
            embedding_matrix[index] = embeddings.get("<unk>", np.zeros(embedding_dim))

    return embedding_matrix

In [ ]:
def load_word2vec_embeddings(file_path, word_to_index, embedding_dim):
    model = KeyedVectors.load_word2vec_format(file_path, binary=True)
    embedding_matrix = np.zeros((len(word_to_index) + 1, embedding_dim))

    for word, index in word_to_index.items():
        embedding_matrix[index] = (
            model[word] if word in model else np.zeros(embedding_dim)
        )

    return embedding_matrix

In [ ]:
def load_fasttext_embeddings(embedding_file, word_index, embedding_dim):
    embeddings_index = {}
    with open(embedding_file, encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype="float32")
            embeddings_index[word] = coefs

    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [ ]:
class Rnn(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size, embedding_dim):
        super(Rnn, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(
            torch.FloatTensor(embedding_matrix), padding_idx=0
        )
        self.rnn = nn.RNN(
            input_size=embedding_dim, hidden_size=hidden_size, batch_first=True
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.rnn(x)
        output = self.fc(output)
        return output

In [ ]:
class Lstm(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size, embedding_dim):
        super(Lstm, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(
            torch.FloatTensor(embedding_matrix), padding_idx=0
        )
        self.lstm = nn.LSTM(
            input_size=embedding_dim, hidden_size=hidden_size, batch_first=True
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out)
        return output

In [ ]:
class Gru(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size, embedding_dim):
        super(Gru, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(
            torch.FloatTensor(embedding_matrix), padding_idx=0
        )
        self.gru = nn.GRU(
            input_size=embedding_dim, hidden_size=hidden_size, batch_first=True
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.gru(x)
        output = self.fc(output)
        return output

In [ ]:
run(
    load_glove_embeddings,
    300,
    "../word_embeddings/glove.6B.300d.txt",
    Rnn,
    "rnn_glove_300",
    30,
)

In [ ]:
run(
    load_glove_embeddings,
    100,
    "../word_embeddings/glove.6B.100d.txt",
    Rnn,
    "rnn_glove_100",
    30,
)

In [ ]:
run(
    load_word2vec_embeddings,
    300,
    "../word_embeddings/GoogleNews-vectors-negative300.bin",
    Rnn,
    "rnn_word2vec",
    20,
)

In [ ]:
run(
    load_fasttext_embeddings,
    300,
    "../word_embeddings/wiki-news-300d-1M.vec",
    Rnn,
    "rnn_fasttext",
    20,
)

In [ ]:
run(
    load_glove_embeddings,
    100,
    "../word_embeddings/glove.6B.100d.txt",
    Lstm,
    "lstm_glove",
    20,
)

In [ ]:
run(
    load_word2vec_embeddings,
    300,
    "../word_embeddings/GoogleNews-vectors-negative300.bin",
    Lstm,
    "lstm_word2vec",
    20,
)

In [ ]:
run(
    load_fasttext_embeddings,
    300,
    "../word_embeddings/wiki-news-300d-1M.vec",
    Lstm,
    "lstm_fasttext",
    20,
)

In [ ]:
run(
    load_glove_embeddings,
    100,
    "../word_embeddings/glove.6B.100d.txt",
    Gru,
    "gru_glove",
    30,
)

In [ ]:
run(
    load_word2vec_embeddings,
    300,
    "../word_embeddings/GoogleNews-vectors-negative300.bin",
    Gru,
    "gru_word2vec",
    30,
)

In [ ]:
run(
    load_fasttext_embeddings,
    300,
    "../word_embeddings/wiki-news-300d-1M.vec",
    Gru,
    "gru_fasttext",
    30,
)